# Dataset Creator
* hiermit die h5py Files bauen und abspeichern

In [1]:
from cnn_utils import prep_data, create_dataset, load_dataset

import os
import cv2

%matplotlib inline

C:\ProgramData\Anaconda3\envs\deeplearning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### read addresses of each image, create labels, shuffle everything and split into train/test

### create HDF-DataFile with empty arrays and labels in preparation for loading the images into it
Takes as input a single path where all labeled images are stored. The label is extracted from their name

In [2]:
# define paths to positive/negative images
input_path_neg = 'first_prototype/neg'
input_path_pos = 'first_prototype/pos'

# Anteile von train/val/test-set hier angeben. Derzeit wird allerdings nur Train/Test-set genutzt
dataSplit = {
    'train_split': 0.9,
    'val_split': 0,
    'test_split': 0.1}

train_addrs, train_labels, test_addrs, test_labels = prep_data(input_path_neg,input_path_pos,dataSplit,shuffle_data = True, datatype = 'png')

### Fill the empty Arrays with image data and save a copy locally

In [3]:
# define where the function stores the hdf5 file
#output_train_path = 'C:/Users/JansPC/AnacondaProjects/Weinseminar/wineModel/datasets/train_dataset.hdf5'
#output_test_path = 'C:/Users/JansPC/AnacondaProjects/Weinseminar/wineModel/datasets/test_dataset.hdf5'

output_train_path = 'datasets/train_128.hdf5'
output_test_path = 'datasets/test_wenig.hdf5'

pixel = {
    "width": 128, 
    "height": 128}

hdf5_train_file, hdf5_test_file = create_dataset(train_addrs, train_labels, test_addrs, test_labels, output_train_path, output_test_path, px = pixel, data_order = 'tf')

## Fill HDF with images
### Train dataset

In [4]:
# a numpy array to save the mean of the images
# mean = np.zeros(train_shape[1:], np.float32)

# loop over train addresses
for i in range(len(train_addrs)):
    # print how many images are saved every 1000 images
    if i % 1000 == 0 and i > 1:
        print ('Train data: {}/{}'.format(i, len(train_addrs)))

    # read an image and resize accordingly
    # cv2 load images as BGR, convert it to RGB
    addr = train_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (pixel['width'], pixel['height']), interpolation=cv2.INTER_CUBIC)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # add any image pre-processing here

    # if the data order is Theano, axis orders should change
    #if data_order == 'th':
    #    img = np.rollaxis(img, 2)

    # save the image and calculate the mean so far
    hdf5_train_file["train_set_x"][i, ...] = img[None]
    #mean += img / float(len(train_labels))

# save the mean and close the hdf5 file
#hdf5_file["train_mean"][...] = mean
hdf5_train_file.close()

Train data: 1000/1146


### Test dataset

In [5]:
# loop over test addresses
for i in range(len(test_addrs)):
    # print how many images are saved every 1000 images
    if i % 1000 == 0 and i > 1:
        print ('Test data: {}/{}'.format(i, len(test_addrs)))

    # read an image and resize to (128, 128)
    # cv2 load images as BGR, convert it to RGB
    addr = test_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (pixel['width'], pixel['height']), interpolation=cv2.INTER_CUBIC)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # add any image pre-processing here

    # if the data order is Theano, axis orders should change
    #if data_order == 'th':
    #    img = np.rollaxis(img, 2)

    # save the image
    hdf5_test_file["test_set_x"][i, ...] = img[None]

# save the mean and close the hdf5 file
#hdf5_file["train_mean"][...] = mean
hdf5_test_file.close()

### Validation dataset

In [ ]:
'''
# loop1 over validation addresses
for i in range(len(val_addrs)):
    # print how many images are saved every 1000 images
    if i % 1000 == 0 and i > 1:
        print 'Validation data: {}/{}'.format(i, len(val_addrs))

    # read an image and resize to (pixel, pixel)
    # cv2 load images as BGR, convert it to RGB
    addr = val_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (pixel, pixel), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # add any image pre-processing here

    # if the data order is Theano, axis orders should change
    if data_order == 'th':
        img = np.rollaxis(img, 2)

    # save the image
    hdf5_file["val_img"][i, ...] = img[None]
'''

In [ ]:
def prep_data(input_path_neg, input_path_pos, dataSplit, shuffle_data = True, datatype = 'jpg'):
    '''
    macht die Vorarbeit für die create_dataset-function. Liest aus den negativ/positiv Ordnern die Bilder raus und erstellt
    labels entsprechend. Teilt dann die Daten je nach Angabe in Trainings/Testdaten ein.

    Argumente:
        input_path_neg      -- Ordner, in dem Bilder der Klasse "negativ" liegen
        input_path_neg      -- Ordner, in dem Bilder der Klasse "positiv" liegen
        dataSplit       -- Dictionary, dass die Split-Anteile für Train/Val/Test Sets enthält
        shuffle_data    -- shuffles the dataset if set on True.

    Returns:
        train_addrs     -- Pfade zu den einzelnen Bildern des trainingssets
        train_labels    -- Array, das trainings labels enthält entsprechend der Reihenfolge in train_addr
        test_addrs      -- Pfade zu den einzelnen Bildern des Testsets
        test_labels     -- Array, das test labels enthält entsprechend der Reihenfolge in test_addr
    '''

    for path,i in input_path_neg,len(input_path_neg):
        
    
    if datatype == 'jpg':
        addrs_neg = glob.glob(input_path_neg + '/*.jpg')
        addrs_pos = glob.glob(input_path_pos + '/*.jpg')

    elif datatype == 'png':
        addrs_neg = glob.glob(input_path_neg + '/*.png')
        addrs_pos = glob.glob(input_path_pos + '/*.png')

    labels_neg = [0 for addr in addrs_neg]
    labels_pos = [1 for addr in addrs_pos]
    
    addrs = addrs_neg + addrs_pos
    labels = labels_neg + labels_pos


    if shuffle_data == True:
        try:
            c = list(zip(addrs, labels))
            shuffle(c)
            addrs, labels = zip(*c)
        except ValueError:
            print ('Es wurden keine labels ausgelesen! Höchstwahrscheinlich stimmt was nicht mit deiner Pfadangabe')
            print ('das hier soll der Pfad zu den neg Bildertn sein' + str(addrs_neg))
            print ('there are ' + str(len(addrs)) + ' adresses')
            print ('there are ' + str(len(labels)) + ' labels')
